In [4]:
import numpy as np
import pandas as pd
from scipy.stats import norm

In [174]:
# Parâmetros do experimento
num_samples = 10 # tamanho da amostra
conf_level = 0.95 # nível de confiança desejado
mean_arrival_time = 1 # taxa de chegada
mean_departure_time = 2 #taxa de saida

In [194]:
print("\nTempo de chegada | Tipo de evento | Número de amostras\n")


n = 0
simul_time = 0
# To calculate waiting line mean time
arrivals = []
waiting_line_time = 0
number_of_arrivals = 0

for _ in range(num_samples):
    arrival_time = np.random.exponential(scale=mean_arrival_time)
    departure_time = np.random.exponential(scale=mean_departure_time)

    if n == 0 or arrival_time < departure_time:
        simul_time += arrival_time
        print(f"{simul_time:8.4f} \t\t{'Chegada':14} {n:2} => {n+1:2}")
        n += 1
        number_of_arrivals+=1
        if (n > 1):
            arrivals.append(simul_time)
    else:
        simul_time += departure_time
        if (len(arrivals) > 0):
            waiting_line_time += simul_time - arrivals[0]
            arrivals.pop(0)
        print(f"{simul_time:8.4f} \t\t{'Saída':14} {n:2} => {n-1:2}")
        n -= 1

mean_waiting_time = waiting_line_time / number_of_arrivals


print(f"\nTotal de pessoas que foram ao banco na simulação: {number_of_arrivals}")
print(f"Número de pessoas na fila ao final da simulação: {n}")
print(f"Tempo total da simulação: {simul_time:.4f}")
print(f"Tempo médio de espera na fila: {(mean_waiting_time):.4f}")



Tempo de chegada | Tipo de evento | Número de amostras

  0.6551 		Chegada         0 =>  1
  0.8934 		Chegada         1 =>  2
  1.6725 		Saída           2 =>  1
  2.5039 		Chegada         1 =>  2
  2.8529 		Chegada         2 =>  3
  3.7442 		Chegada         3 =>  4
  4.1741 		Saída           4 =>  3
  4.8520 		Saída           3 =>  2
  5.2796 		Chegada         2 =>  3
  7.3016 		Chegada         3 =>  4

Total de pessoas que foram ao banco na simulação: 7
Número de pessoas na fila ao final da simulação: 4
Tempo total da simulação: 7.3016
Tempo médio de espera na fila: 0.6355


In [177]:
def simulador(num_samples, mean_arrival_time, mean_departure_time):
    n = 0
    simul_time = 0
    # To calculate waiting line mean time
    arrivals = []
    waiting_line_time = 0
    number_of_arrivals = 0

    for _ in range(num_samples):
        arrival_time = np.random.exponential(scale=mean_arrival_time)
        departure_time = np.random.exponential(scale=mean_departure_time)

        if n == 0 or arrival_time < departure_time:
            simul_time += arrival_time
            # print(f"{simul_time:8.4f} \t\t{'Chegada':14} {n:2} => {n+1:2}")
            number_of_arrivals+=1
            n += 1
            if (n > 1):
                arrivals.append(simul_time)
        else:
            simul_time += departure_time
            if (len(arrivals) > 0):
                waiting_line_time += simul_time - arrivals[0]
                arrivals.pop(0)
            # print(f"{simul_time:8.4f} \t\t{'Saída':14} {n:2} => {n-1:2}")
            n -= 1
    
    pessoa_fila = n
    tempo_medio_espera = waiting_line_time/number_of_arrivals
    return pessoa_fila, simul_time, tempo_medio_espera

In [178]:
import math
import scipy.stats as st

In [173]:
st.norm.cdf(0.95)
st.norm.ppf(.95)


1.6448536269514722

In [179]:
print("\nTempo de chegada | Tipo de evento | Número de amostras\n")


n = 0
simul_time = 0
# To calculate waiting line mean time
arrivals = []
waiting_line_time = 0
number_of_arrivals = 0

for _ in range(num_samples):
    arrival_time = np.random.exponential(scale=mean_arrival_time)
    departure_time = np.random.exponential(scale=mean_departure_time)

    if n == 0 or arrival_time < departure_time:
        simul_time += arrival_time
        # print(f"{simul_time:8.4f} \t\t{'Chegada':14} {n:2} => {n+1:2}")
        n += 1
        number_of_arrivals+=1
        if (n > 1):
            arrivals.append(simul_time)
    else:
        simul_time += departure_time
        if (len(arrivals) > 0):
            waiting_line_time += simul_time - arrivals[0]
            arrivals.pop(0)
        # print(f"{simul_time:8.4f} \t\t{'Saída':14} {n:2} => {n-1:2}")
        n -= 1

mean_waiting_time = waiting_line_time / number_of_arrivals


print(f"\nTotal de pessoas que foram ao banco na simulação: {number_of_arrivals}")
print(f"Número de pessoas na fila ao final da simulação: {n}")
print(f"Tempo total da simulação: {simul_time:.4f}")
print(f"Tempo médio de espera na fila: {(mean_waiting_time):.4f}")



Tempo de chegada | Tipo de evento | Número de amostras


Total de pessoas que foram ao banco na simulação: 8
Número de pessoas na fila ao final da simulação: 6
Tempo total da simulação: 4.9021
Tempo médio de espera na fila: 0.5576


In [185]:
def simulator_confidence_interval(sample_size, confidence_level):
    """
    Simula o processo de fila m/m/1 um intervalo de confiança para tempo médio de espera na fila.

    sample_size: tamanho da amostra para a simulação
    confidence_level: nível de confiança desejado para o intervalo de confiança

    Retorna um intervalo de confiança para o tempo médio de espera na fila.
    """
    results = [simulador(120, 1, 2)[2] for _ in range(sample_size)]


    # Cálculo do intervalo de confiança
    # proportion = np.sum(results) / sample_size
    # std_error = math.sqrt(proportion * (1 - proportion) / sample_size)
    # margin_error = std_error * st.norm.ppf(confidence_level)
    # lower_bound = proportion - margin_error #limite superior do IC
    # upper_bound = proportion + margin_error  # limite inferior do IC


    # Outro calculo
    x_barra = np.sum(results) / sample_size
    z = 1.96
    s = np.std(results)
    limite_inferior = x_barra - z*s/math.sqrt(sample_size)
    limite_superior = x_barra + z*s/math.sqrt(sample_size)

    # print(limite_inferior, limite_superior)
    return (limite_inferior, limite_superior), results
    return (lower_bound, upper_bound), results

intervalo, data = simulator_confidence_interval(1000, 0.95)

print(intervalo)


(9.436801807657782, 9.726652416595497)


In [130]:
import plotly.express as px

In [186]:
max(data)

19.651182223040767

In [193]:
px.histogram(data, nbins=50)

In [70]:
df = pd.DataFrame(data, columns=['Tempo de Chegada', 'Tipo de Evento', 'Antes', 'Depois'])
df

,Tempo de Chegada,Tipo de Evento,Antes,Depois
0,0.9743,Chegada,0,1
1,1.1336,Chegada,1,2
2,1.9404,Chegada,2,3
3,1.9802,Saída,3,4
4,2.5780,Saída,2,3
...,...,...,...,...
115,61.1757,Saída,1,2
116,61.6270,Chegada,0,1
117,61.8587,Chegada,1,2
118,63.4419,Saída,2,3
